<a href="https://www.kaggle.com/code/waechter/identify-contrail-submit-stack?scriptVersionId=138099749" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture
import contrails_utils as cu

In [2]:
print(cu.Commun.colors.GREEN.format("contrails_utils imported"))

contrails_utils imported


In [3]:
# %%capture
import os

import numpy as np
import pandas as pd

import yaml
from pathlib import Path

import torch

In [4]:
### Kaggle environment variable
INTERACTIVE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive"
RERUN = os.getenv("KAGGLE_IS_COMPETITION_RERUN") is not None
print(f"Interactive {INTERACTIVE}, RERUN {RERUN}")

Interactive True, RERUN False


In [6]:
config = cu.PyLModel.CFG.default_predict_config(image_size=384)
# or config = PyLModel.CFG.open_config(CFG_PATH)

In [7]:
N_TIMES_BEFORE = 4

data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')
data_test = data_path / 'test'
submission = pd.read_csv(data_path /'sample_submission.csv', index_col='record_id')

In [8]:
test_df = pd.DataFrame([{'record_id': path_test.name, 'path':path_test, "sequence_num":N_TIMES_BEFORE} for path_test in data_test.iterdir()])
# test_df

# Submission 

In [13]:
%%time
paths_model= list(Path("/kaggle/input/identify-contrails-models").glob("**/*.ckpt"))+list(Path("/kaggle/input/model-weights").glob("*.ckpt"))+list(Path("/kaggle/input/training-with-4-folds/models").glob("*.ckpt"))
high_threshold, low_threshold = (0.45, 0.3)
all_preds, models_len = cu.PyLModel.predict_with_models(test_df, models=paths_model, config=config)

Predict with model: model-seq0-val_dice0.6326.ckpt
Predict with model: pb_model_dice_649.ckpt
Predict with model: model_v13.ckpt
Predict with model: model_v9.ckpt
Predict with model: model_v11.ckpt
Predict with model: model_improved.ckpt
Predict with model: model-f1-val_dice=0.6673.ckpt
Predict with model: model-f3-val_dice=0.6594.ckpt
Predict with model: model-f2-val_dice=0.6617.ckpt
Predict with model: model-f0-val_dice=0.6616.ckpt
CPU times: user 16.9 s, sys: 3.95 s, total: 20.8 s
Wall time: 39.9 s


In [10]:
model_importance = {
    "model-seq0-val_dice0.6326":1,
    "Predict with model: pb_model_dice_649":4,
    "Predict with model: model_v13":1,
    "Predict with model: model_v9":1,
    "Predict with model: model_v11":1,
    "Predict with model: model_improved":1,
    "Predict with model: model-f1-val_dice=0.6673":5,
    "Predict with model: model-f3-val_dice=0.6594":5,
    "Predict with model: model-f2-val_dice=0.6617":5,
    "Predict with model: model-f0-val_dice=0.6616":5,
}
model_individual_threshold = {
    "model-seq0-val_dice0.6326":(0.35,0.45),
    "Predict with model: pb_model_dice_649":(0.25,0.45),
    "Predict with model: model_v13":(0.25,0.45),
    "Predict with model: model_v9":(0.25,0.45),
    "Predict with model: model_v11":(0.25,0.45),
    "Predict with model: model_improved":(0.25,0.45),
    "Predict with model: model-f1-val_dice=0.6673":(0.35,0.45),
    "Predict with model: model-f3-val_dice=0.6594":(0.35,0.45),
    "Predict with model: model-f2-val_dice=0.6617":(0.35,0.45),
    "Predict with model: model-f0-val_dice=0.6616":(0.35,0.45),
}

In [14]:
%%time
for index in submission.index.tolist():
    predicted_mask_with_threshold = cu.PyLModel.stack_prediction_mask_threshold(all_preds, index, high_threshold, low_threshold, models_len, vote_majority=0.35, model_importance=model_importance, model_individual_threshold=model_individual_threshold)
    submission.loc[int(index), 'encoded_pixels'] = cu.Commun.run_length_encode(predicted_mask_with_threshold)

CPU times: user 42.2 ms, sys: 4.4 ms, total: 46.6 ms
Wall time: 49.9 ms


/kaggle/usr/lib/contrails_utils/contrails_utils.py:116: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3571.)
  dots = np.where((x.T.flatten() == fg_val_min))[0]


In [15]:
submission.to_csv('submission.csv')
submission

,encoded_pixels
record_id,
1000834164244036115,-
1002653297254493116,-
